In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

batch_size = 7
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.ToTensor()),
        batch_size = batch_size, shuffle=True)

from random import randint

print("kk")


##### TODO ######
# Circuit picking (all the reshapes and stuff)
# Circuit dropout
# Initial connection

kk


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if device != 'cpu':
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

num_classes=10

In [3]:
class CircuitNet(nn.Module):
    def __init__(self,circuit_dropout=0.5,circuit_ban_rate = 0.3,num_classes=12):
        super(CircuitNet, self).__init__()
        
        self.circuit_ban_rate = circuit_ban_rate
        self.circuit_dropout = circuit_dropout
        self.state_size = 100
        self.num_lstm_layers = 3
        
        #monitor use of each circuit to balance training
        self.used_circuits = np.zeros([40,2])
        for i in range(40):
            self.used_circuits[i][0] = i
        
        #encode to be fed in to lstm
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 16, 3, stride=2, padding=1),  
            nn.ReLU(True),
            nn.BatchNorm1d(16),
            nn.MaxPool1d(2, stride=2),  
            nn.Conv1d(16, 8, 3, stride=2, padding=1),  
            nn.ReLU(True),
            nn.BatchNorm1d(8),
            nn.MaxPool1d(2, stride=2)
        )
        
        
        
        self.lstm = nn.LSTM(8*49,self.state_size, self.num_lstm_layers)
        self.hidden = self.init_hidden()
        
        
        self.circuits_to_use = 10
        self.n_circuits = 20
        self.circuit_in_dim = 25
        
        #decode latent lstm into circuits
        self.decoder = nn.Sequential(
            nn.Linear(self.state_size, 400),
            nn.ReLU(True),
            nn.Linear(400, (self.circuits_to_use * self.circuit_in_dim) + 
                      (self.n_circuits *  self.circuits_to_use)),
            nn.ReLU(True)
            
        )
        
        #self.dense_out = nn.Sequential(
        #    nn.Linear(self.state_size, 10),
        #    nn.ReLU(True))
        #decoder shape = [batch_size, n_circuits, circuit_dim,]
        
        
        self.circuits = []
        #define circuits
        for i in range(self.n_circuits):
        
            
            self.circuits.append(nn.Sequential(
                nn.Conv1d(1, 8, kernel_size=5, stride=1, padding=2),
                nn.BatchNorm1d(8),
                nn.ReLU(True),
                nn.MaxPool1d(kernel_size=2, stride=2),
            
                nn.Conv1d(8, 16, kernel_size=5, stride=1, padding=2),
                nn.BatchNorm1d(16),
                nn.ReLU(True),
                nn.MaxPool1d(kernel_size=2, stride=2)))
        
        #bridge concat of circuits into next encoder
        self.circuit_bridge = nn.Sequential(
            nn.Linear(960, 784),
            nn.ReLU(True))
        
        #final circuit to prediction
        self.output = nn.Sequential(
            nn.Linear(16*60, 400),
            nn.ReLU(True),
            nn.Linear(400, 10))
        
    #init hidden state for lstm layer
    def init_hidden(self):
        return (torch.zeros(self.num_lstm_layers,1, self.state_size, device=device),
                torch.zeros(self.num_lstm_layers,1, self.state_size, device=device))
    
    def connection(self, input, verbose):
        encoder_ = self.encoder(input)
        encoder_flat = encoder_.view(encoder_.size(0),1, -1)
        print(encoder_flat.shape)
        
        lstm_out, self.hidden = self.lstm(encoder_flat,self.hidden)
        
        decoder = self.decoder(lstm_out)
        
        circuits_to_use = decoder[:,:,0:self.n_circuits * self.circuits_to_use].view(decoder.size(0),
                                                                                        self.circuits_to_use,
                                                                                        self.n_circuits)
        data_to_send = decoder[:,:,self.n_circuits * self.circuits_to_use:].view(decoder.size(0),
                                                                                        self.circuits_to_use,
                                                                                        self.circuit_in_dim)
        
        #ban random entire circuits
        #bans = np.random.randint(self.n_circuits,size=int(self.circuit_ban_rate*self.n_circuits))
        bans = self.used_circuits[self.n_circuits-6:self.n_circuits]
        circuit_out = []
        for i in range(self.circuits_to_use):
            #pick circuit from argmax
            circuit_idx = circuits_to_use[:,i,:].max(1)[1]
            if(verbose):
                print("max : " + str(circuit_idx))
            
            batch = []
            for b in range(batch_size):
                
                
                circuit = self.circuits[circuit_idx[b]](data_to_send[b][i].unsqueeze(0).unsqueeze(0))
                self.used_circuits[circuit_idx[b]] += 1
                
                ## apply dropout
                if randint(1,100) < self.circuit_dropout*100 or circuit_idx[b] in bans:
                    circuit = torch.zeros(circuit.shape,device=device)  
                
                    
                batch.append(circuit)
            batch = torch.cat(batch,0)
            circuit_out.append(batch)
        
        self.used_circuits[:,1].sort(axis=0)
        
        circuit_out = torch.cat(circuit_out,2)
        circuit_out_flat = circuit_out.view(circuit_out.size(0), -1)
        return circuit_out_flat
        
    def forward(self, x, verbose):
        x_flat = x.view(x.size(0),1, -1)
        
        #######
        connect = self.connection(x_flat, verbose)
        #bridge = self.circuit_bridge(connect).unsqueeze(1)
        #print(bridge.shape)
        #connect1 = self.connection(bridge, verbose)
        
        ######
        
        out = self.output(connect)
        return out
        

if device != 'cpu':
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
model = CircuitNet(circuit_dropout=0.4, circuit_ban_rate = 0.6).to(device)
torch.set_default_tensor_type('torch.FloatTensor')

for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    output = model(data,False)
    break

torch.Size([7, 1, 392])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])


In [4]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model.train()
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    verbose = False
    if(batch_idx % 100 == 0):
        verbose = False
    output = model(data,verbose)
    loss =  nn.CrossEntropyLoss()(output, target)
    #loss = (output-target)**2
    loss.backward(retain_graph=True)
    optimizer.step()
    
    model.hidden = model.init_hidden()
    if(batch_idx % 100 == 0):
        print("Index : {}  Loss : {:.5f}  Real : {}  Pred : {}".format(batch_idx,loss[0],target[0:4],output[0:4].max(1)[1]))

Index : 0  Loss : 2.28709  Real : tensor([ 6,  7,  8,  6], device='cuda:0')  Pred : tensor([ 6,  0,  4,  1], device='cuda:0')


/home/liam/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Index : 100  Loss : 2.37416  Real : tensor([ 2,  9,  6,  4], device='cuda:0')  Pred : tensor([ 4,  4,  7,  7], device='cuda:0')
Index : 200  Loss : 2.32262  Real : tensor([ 2,  1,  7,  0], device='cuda:0')  Pred : tensor([ 4,  4,  4,  8], device='cuda:0')
Index : 300  Loss : 2.37367  Real : tensor([ 4,  8,  0,  6], device='cuda:0')  Pred : tensor([ 8,  8,  1,  1], device='cuda:0')
Index : 400  Loss : 2.37504  Real : tensor([ 0,  2,  9,  5], device='cuda:0')  Pred : tensor([ 7,  0,  7,  0], device='cuda:0')
Index : 500  Loss : 2.31267  Real : tensor([ 1,  5,  1,  8], device='cuda:0')  Pred : tensor([ 0,  6,  0,  0], device='cuda:0')
Index : 600  Loss : 2.27794  Real : tensor([ 8,  7,  8,  8], device='cuda:0')  Pred : tensor([ 1,  8,  1,  7], device='cuda:0')
Index : 700  Loss : 2.34781  Real : tensor([ 9,  3,  1,  2], device='cuda:0')  Pred : tensor([ 7,  0,  3,  7], device='cuda:0')
Index : 800  Loss : 2.35030  Real : tensor([ 4,  1,  9,  2], device='cuda:0')  Pred : tensor([ 7,  1,  7

Index : 6500  Loss : 0.42022  Real : tensor([ 9,  7,  2,  4], device='cuda:0')  Pred : tensor([ 9,  7,  2,  4], device='cuda:0')
Index : 6600  Loss : 0.21539  Real : tensor([ 7,  6,  1,  4], device='cuda:0')  Pred : tensor([ 7,  6,  1,  4], device='cuda:0')
Index : 6700  Loss : 2.07729  Real : tensor([ 5,  6,  8,  2], device='cuda:0')  Pred : tensor([ 3,  8,  8,  0], device='cuda:0')
Index : 6800  Loss : 0.82994  Real : tensor([ 5,  9,  6,  5], device='cuda:0')  Pred : tensor([ 7,  9,  6,  3], device='cuda:0')
Index : 6900  Loss : 0.64336  Real : tensor([ 6,  2,  4,  2], device='cuda:0')  Pred : tensor([ 6,  2,  4,  2], device='cuda:0')
Index : 7000  Loss : 1.28542  Real : tensor([ 2,  3,  8,  7], device='cuda:0')  Pred : tensor([ 2,  8,  0,  7], device='cuda:0')
Index : 7100  Loss : 0.56360  Real : tensor([ 3,  2,  3,  6], device='cuda:0')  Pred : tensor([ 8,  2,  3,  6], device='cuda:0')
Index : 7200  Loss : 0.72517  Real : tensor([ 4,  3,  8,  0], device='cuda:0')  Pred : tensor([ 4

IndexError: index 3 is out of bounds for dimension 0 with size 3